In [1]:
import pandas as pd
import numpy as np

import scipy.io
import glob
import os

#models
from smlr import SMLR as smlr
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import top_k_accuracy_score


In [2]:
#Predefine the used layers, imagenet ids, and subjects
layers = ['conv1', 'conv2', 'conv3','conv4', 'conv5', 'fc6', 'fc7', 'fc8']
subjs = ['Subj1', 'Subj2', 'Subj3', 'Subj4', 'Subj5']
rois = ['VC', 'V1', 'V2', 'V3', 'V4', 'PPA', 'LOC', 'FFA']

#Key dataframe converting stimulus id to imagenet id
key = pd.read_csv('Intersecting image.csv')
Imagenet_id = key['Imagenet_id']

#Misc settings
random_seed = 2861
n_iter = 200
sli = pd.IndexSlice

In [3]:
def data_extractor(roi, p, key=key, layers=layers, subjs=subjs):
    '''
    Function to extract data from .mat files of a single roi in all subjects
    and combine them into a single dataframe
    ---
    roi: the region of interest. case sensitive to the filename in your work directory
    p: path to the directory of subject folders
    key: dataframe containing the key mapping of included alexnet_id to imagenet_id
    ---
    returns combined dataframe
    '''
    df = pd.DataFrame()

    for mod in layers:
        for subj in subjs:
            #Path construction
            path = p + mod + "/" + subj + "/" + roi + "/"
            #bfd\best_features\1000\conv1\Subj1\VC
            files = glob.glob(os.path.join(path, "*.mat"))
        
            #Create a tmp df containing all mat data in that folder
            tmp = pd.concat([pd.DataFrame(scipy.io.loadmat(f)['best_feature_data']) for f  in files]
                         , ignore_index = True)
    
            #Add a new column to label the entries with the model name, subject number and Alexnet id
            tmp.insert(0, 'layer', mod)
            tmp.insert(0, 'Subject', subj)
            
            #Create a pd df for the index values, transforming Alexnet_id in the process
            #e.g. from n01443537_22563 to 1443537
            l = pd.Series([0]*len(files), dtype='int')
            for i in range(len(files)):
                l.iloc[i] = os.path.basename(files[i]).split('_')[0][2:]
            l = np.repeat(l, 35)
            
            #Repeat the values 35 times, corresponding to 35 samples per image
            l= pd.DataFrame(l, columns = ['Alexnet_id'])
            tmp = pd.concat([l.reset_index(drop=True), tmp], axis = 1, ignore_index = True)

            #Concatenate the resultant dfs together
            df = pd.concat([df, tmp], axis = 0, ignore_index = True)
    #Transform the key into a dictionary to use it in the next step
    key.Alexnet_id = key.Alexnet_id.astype('str')
    key = key.set_index('Alexnet_id')['Imagenet_id'].to_dict()
    
    #Name the metadata columns as appropriate
    df = df.rename(columns = {0 : 'Alexnet_id', 1: 'Subject', 2: 'Layer'})
    
    #Create a new column, Imagenet_id in the master df, by mapping the dic with the Alexnet id values
    df.insert(0, 'Imagenet_id', df.Alexnet_id.map(key))
    
    #Drop rows with nan values (these will have no associated Imagenet_id in the key. Therefore, they are not included)
    df.dropna(inplace = True)
    
    #Drop the Alexnet_id column, and set the Imagenet_id column to int
    df.drop(['Alexnet_id'], axis = 1, inplace = True)
    df['Imagenet_id'] = df['Imagenet_id'].astype('int')
    
    return df.reset_index(drop = True)

In [4]:
def create_multi_index(x, indices):
    '''
    Duplicate multiple lists sequentially to be used in a multi-index dataframe
    e.g.: 
    list = [1, 2, 3]
    after broadcasting onto a 9-row dataframe,
    list = [1, 2, 3, 1, 2, 3, 1, 2, 3]
    ----
    x: number of rows in a dataframe. Use x = 1 to calculate it by multiplying the len of lists together
    indices: list (of lists) to be duplicated to match length x.
    '''
    #x is the number of rows in the dataframe; calculated by multiplying the len of lists together, or provided beforehand
    if x == 1:
        for i in range(len(indices)):
            x *= len(indices[i])
    mul_index = []
    
    for index in indices:
        y = x / len(index)
        index = list(index) * int(y)
        mul_index += [index]
    return mul_index

We have 5 subjects, with 10 ROIs per subject. Each ROI has voxel data that will be used in our experiment.
The subjects underwent image training, followed by a perception test and an imagery test. Only the perception test data will be used in this experiment.

The participants were shown 35 images for the test. Only 9 images will be selected out of those, and used for the experiment. Those were selected because their categories intersect with the ImageNet library.

Pseudocode:
1. Extract voxel, stimulus id, and data type data from each subject.
2. Remove the training and imagery test data.
3. Set the index to be the stimulus id, and join the dataframe with the stimulus id - imagenet id pairings.
4. Drop rows with no image id, and drop the stimulus id, and data type columns.
5. Split into training/test data, run Smlr and SVM, and calculate the prediction percentage.
6. Repeat the experiment on all ROIs, and display the results in a table.

In [5]:
#Import the pref images vectors into a single dataframe
pref_img = pd.DataFrame()
for mod in layers:
    #Path in my work directory, replace with yours as necessary
    path = 'bfd/best_features_pref_images/best_features/1000/' + mod + "/"
    files = glob.glob(os.path.join(path, "*.mat"))
        
    #Create a tmp df containing all mat data in that folder, then use the index to create the imagenet_id column; same values
    tmp = pd.concat([pd.DataFrame(scipy.io.loadmat(f)['best_feature_data']).T for f in files]
                         , ignore_index = True).reset_index().rename(columns = {"index":'Imagenet_id'})
    
    #Add a new column to label the entries with the model name
    tmp.insert(0, 'layer', mod)
    
    #Concatenate the resultant dfs together
    pref_img = pd.concat([pref_img, tmp], axis = 0, ignore_index = True)

In [6]:
#Perform inner join on key and pref_img dataframes, removing non-intersecting images
pref_img = key.set_index('Imagenet_id').join(pref_img.set_index('Imagenet_id')).reset_index()

#Drop unnecessary columns to prepare the dataframe to be used
pref_img = pref_img.drop(['Image_description', 'Alexnet_id'], axis = 1)

#pref_img df now is 168 entries (21*8) by 1000 features + 2 metadata columns
pref_img

,Imagenet_id,layer,0,1,2,3,4,5,6,7,...,990,991,992,993,994,995,996,997,998,999
0,1,conv1,11.476718,-25.895676,-40.893665,10.021077,10.557981,-36.217346,-41.286446,-75.275200,...,33.396690,82.087357,-12.189371,-119.277802,20.036339,30.214413,-13.125332,-5.200476,-6.953192,37.496685
1,1,conv2,-262.386475,-91.425667,-236.797638,-109.876915,-132.229218,-141.268829,-87.422348,-131.869278,...,-184.656128,-181.334778,-62.832211,-155.209991,12.391937,-168.993835,-151.324615,-73.859734,-218.618866,-86.510498
2,1,conv3,113.617676,95.916077,244.712173,1.555827,-17.988266,17.713959,154.402344,-196.257004,...,-19.722656,95.950569,-38.018047,-34.416386,-34.806896,1.846160,-36.299171,10.153909,-7.466252,-87.169029
3,1,conv4,98.402870,2.452147,61.812294,-23.877178,-32.050632,22.487713,-20.046038,28.367855,...,26.840666,-15.808105,-35.321037,22.377811,-1.256709,33.592144,18.019812,-34.405991,-0.702034,-62.810215
4,1,conv5,-64.292816,-50.911831,-24.796650,-57.435677,-65.899101,-42.336655,-21.849260,-72.391479,...,-39.734768,-26.043579,-61.206161,-32.466904,-58.548798,-80.592941,0.572763,-53.210026,11.932649,-37.241249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,897,conv4,78.928543,-66.840408,35.047390,-68.855141,-13.382402,-11.286433,-18.747446,-28.757263,...,-50.204189,9.942905,-13.083759,-17.605408,-13.542098,-48.618340,-8.268844,-69.221214,-25.744562,16.998856
164,897,conv5,-23.527922,-31.505159,-40.243137,-35.339340,-64.311089,-18.807806,5.673960,-23.250013,...,-63.854099,-12.565029,-36.956093,-51.224533,-42.695961,-13.396852,-62.568878,-81.089882,-69.127541,-26.845081
165,897,fc6,-52.567741,-6.406816,-10.177212,-35.640720,-62.432510,-3.436984,-18.650530,-30.454874,...,-33.538307,-6.807866,-39.883553,-82.117966,-7.156143,-45.256710,-34.827076,1.890019,-35.873177,-10.288158
166,897,fc7,0.133029,-7.150592,-3.432850,-11.783286,-8.743981,-23.279251,-8.044333,-11.834446,...,-21.521461,5.955750,-2.785596,10.657142,-7.704358,-7.466293,-4.669672,-1.998870,7.070156,31.475857


In [ ]:
#Added fixer to properly construct the indices. I don't know how to fix it properly. 
#I think having rois and layers with the same number of items per list is the source of the problem. I don't know why.

#When the length of ROI and layer lists is the same, the multi-index 
#is arranged so that each ROI has 8 copies of the same layer instead of 1 copy of each layer. Sorting the index
#does not solve this problem.

rois = ['VC', 'V1', 'V2', 'V3', 'V4', 'PPA', 'LOC', 'FFA', 'fixer'] 

sum_index = create_multi_index(x = 1, indices = [rois, layers])
sum_col = ['Top 1% Accuracy', 'Top 5% Accuracy', 'Corr-coef'] 

#df to contain bfd methods' results
summary_bfd = pd.DataFrame(index = sum_index, columns = sum_col).sort_index().drop('fixer')

sum_col = ['1% SVM', '5% SVM', 'Corr-SVM', '1% logreg', '5% logreg', 'Corr-logreg']
sum_index = create_multi_index(x = 1, indices = [rois, layers, subjs])

#df to contain other methods' results
summary_others = pd.DataFrame(index = sum_index, columns = sum_col).sort_index().drop('fixer')

In [ ]:
#Calculating top 1% accuracy, top 5% accuracy, and corr coef for the BFD method and others per roi:
for roi in rois:
    
    #Fix for the problem mentioned above.
    if roi == 'fixer':
        break
        
    roi_df = data_extractor(roi = roi, p = 'bfd/best_features/1000/', key = key, layers = layers, subjs = subjs)
    
    
    
    for layer in layers:
            for subj in subjs:
                
                df = roi_df[(roi_df.Subject == subj) & (roi_df.Layer == layer)]
                #Split data into training and test data for the models
                #test_size was selected to be 5 repetitions for each image (5 * 21 = 105)
                X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 3:], df.iloc[:, 0], test_size = 105, random_state = random_seed)
    
                #SVM
                svm = SVC(kernel = 'poly', probability = True)
                svm.fit(X_train, y_train)
                svm_y_score = svm.predict_proba(X_test)
                
                #Results
                summary_others.loc[roi, layer, subj]['Corr-SVM'] = np.corrcoef(y_test, svm.predict(X_test))[0][1]
                summary_others.loc[roi, layer, subj]['1% SVM'] = top_k_accuracy_score(y_test, svm_y_score, k = 1)
                summary_others.loc[roi, layer, subj]['5% SVM'] = top_k_accuracy_score(y_test, svm_y_score, k = 5)
    
                #LogReg
                model = LogisticRegression(penalty = 'l1', solver = 'saga', random_state = random_seed, max_iter = n_iter)
                model.fit(X_train, y_train)    
                model_y_score = model.predict_proba(X_test)
                #Results
                summary_others.loc[roi, layer, subj]['Corr-logreg'] = np.corrcoef(y_test, model.predict(X_test))[0][1]
                summary_others.loc[roi, layer, subj]['1% logreg'] = top_k_accuracy_score(y_test, model_y_score, k = 1)
                summary_others.loc[roi, layer, subj]['5% logreg'] = top_k_accuracy_score(y_test, model_y_score, k = 5)
    
    
    
    #BFD method
    corr_roi = np.corrcoef(pref_img.iloc[:, 2:].astype('float'),
                           roi_df.iloc[:, 3:].astype('float'))
    multi_index = [np.array(roi_df['Imagenet_id']),
                   np.array(roi_df['Layer']),
                   np.array(roi_df['Subject'])]
    multi_column = [np.array(pref_img.Imagenet_id),
                    np.array(pref_img.layer)]
    
    #Running the corrcoef on whole dfs and slicing it this way extracts all the data from the correlation matrix
    #Meaning we don't need to use for loops. I confirmed this through experimentation (on VC only).
    corr_roi = pd.DataFrame(corr_roi[:168, 168:].T, index = multi_index, columns = multi_column)
    
    columns = ['is in Top-1?', 'is in Top-5?', 'Corr-coef']
    index = create_multi_index (x = roi_df.shape[0]/35, indices = [Imagenet_id, layers, subjs])
    results = pd.DataFrame(index = index, columns=columns).sort_index()

    #Calculate the results for the roi, and put them in the summary dataframe
    for idx in Imagenet_id:
        for layer in layers:
            for subj in subjs:
                results.loc[:,'is in Top-1?'][idx][layer][subj] = idx in (corr_roi.loc[:, idx][layer][sli[:, layer, subj]].sort_values(ascending=False)[:1])
        
                results.loc[:,'is in Top-5?'][idx][layer][subj] = idx in (corr_roi.loc[:, idx][layer][sli[:, layer, subj]].sort_values(ascending=False)[:5])
            
                results.loc[:, 'Corr-coef'][idx][layer][subj] = corr_roi.loc[:, idx][layer][idx][layer][subj].mean()
    
    for layer in layers:
        summary_bfd.loc[roi].loc[ layer,'Top 1% Accuracy'] = (results.loc[sli[:, layer, :] , 'is in Top-1?'].sum()/
                                                       results.loc[sli[:, layer, :] , 'is in Top-1?'].count())
        summary_bfd.loc[roi].loc[ layer,'Top 5% Accuracy'] = (results.loc[sli[:, layer, :] , 'is in Top-5?'].sum()/
                                                       results.loc[sli[:, layer, :] , 'is in Top-5?'].count())
        summary_bfd.loc[roi].loc[ layer, 'Corr-coef'] = results.loc[sli[:, layer, :], 'Corr-coef'].mean()

In [ ]:
#Save the results.
summary_bfd.reset_index().to_csv('bfd.csv')
summary_others.reset_index().to_csv('others.csv')